In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import umap
from sklearn.preprocessing import PowerTransformer
from tqdm.notebook import tqdm, trange
from sklearn.metrics import davies_bouldin_score
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from prince import FAMD, PCA
from sklearn.preprocessing import StandardScaler
from kmodes.kprototypes import KPrototypes
# import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore")


#Our own librabries
import sys
sys.path.insert(1, '../../Environnement/')
from utils import clean_data, add_Loss
import clean
import merge

In [5]:
#Select the dataset of one season of one year
YEAR = 2019
SEASON = "Rabi" # or "Kharif" 

#Path to the dataset
pathData_R = f"..\..\Data\RawDataUnified\RawData_{YEAR}_Rabi"
pathData_K= f"..\..\Data\RawDataUnified\RawData_{YEAR}_Kharif"



df_R = pd.read_csv(pathData_R)
df_K=pd.read_csv(pathData_K)

In [4]:
#Clean an add loss to data
df_R=add_Loss(clean_data(df_R))
df_K=add_Loss(clean_data(df_K))

df_R.head()

,Crop,Area Sown (Ha),Area Insured (Ha),SI Per Ha (Inr/Ha),Lp_2011,Lp_2012,Lp_2013,Lp_2014,Lp_2015,Lp_2016,Lp_2017,Loss
key,,,,,,,,,,,,
andhra pradesh_anantapur_vidapanakal___,Bengal Gram (Chana),179.859471,160.816527,30000.0,0.014822,0.0,0.0,0.000000,0.882341,0.645266,0.000000,1.078638e+05
andhra pradesh_anantapur_vajrakarur___,Bengal Gram (Chana),179.859471,160.816527,30000.0,0.000000,0.0,0.0,0.000000,0.217446,0.000000,0.347871,0.000000e+00
andhra pradesh_anantapur_gooty___,Bengal Gram (Chana),179.859471,160.816527,30000.0,0.330642,0.0,0.0,0.086273,0.000000,0.496653,0.122131,1.516659e+06
andhra pradesh_anantapur_guntakal___,Bengal Gram (Chana),179.859471,160.816527,30000.0,0.330642,0.0,0.0,0.086273,0.000000,0.496653,0.122131,1.516659e+06
andhra pradesh_anantapur_pamidi___,Bengal Gram (Chana),179.859471,160.816527,30000.0,0.330642,0.0,0.0,0.086273,0.000000,0.496653,0.122131,1.516659e+06


In [10]:
print(df_R['State'].unique())
print(df_K['State'].unique())
print()
print(set(df_R['State'].unique())-set(df_K['State'].unique()))

['Andhra Pradesh' 'Chhattisgarh' 'Gujarat' 'Haryana' 'Karnataka'
 'Madhya Pradesh' 'Maharashtra' 'Odisha' 'Rajasthan' 'Tamil Nadu'
 'Telangana' 'Uttar Pradesh' 'West Bengal']
['Andhra Pradesh' 'Chhattisgarh' 'Gujarat' 'Haryana' 'Jharkhand'
 'Karnataka' 'Madhya Pradesh' 'Maharashtra' 'Odisha' 'Rajasthan'
 'Telangana' 'Uttar Pradesh' 'Uttarakhand' 'West Bengal']

{'Tamil Nadu'}


In [32]:
all_states = ['andhra pradesh','chhattisgarh','gujarat','haryana','jharkhand','karnataka','madhya pradesh','maharashtra',
              'odisha','rajasthan','telangana','uttar pradesh','uttarakhand','west bengal','tamil nadu','assam']
print(len(all_states))

f = lambda stri : stri.lower()
states_R = list(map(f, df_R['State'].unique()))
print(len(states_R))
states_K = list(map(f,df_K['State'].unique()))
print(len(states_K))

print(set(all_states)- set(states_R))
print(set(all_states)- set(states_K))

16
13
14
{'assam', 'uttarakhand', 'jharkhand'}
{'assam', 'tamil nadu'}


In [55]:
YEAR = 2018

pathData_R = f"..\..\Data\RawDataUnified\RawData_{YEAR}_Rabi"
pathData_K= f"..\..\Data\RawDataUnified\RawData_{YEAR}_Kharif"


states_compl_R = ['Assam', 'Uttarakhand', 'Jharkhand']
df_compl_R = pd.read_csv(pathData_R)


states_compl_K = ['Assam', 'Tamil Nadu']
df_compl_K = pd.read_csv(pathData_K)
df_compl_K['State'].unique()

array(['Andhra Pradesh', 'Bihar', 'Chhattisgarh', 'Gujarat', 'Haryana',
       'Jharkhand', 'Karnataka', 'Madhya Pradesh', 'Maharashtra',
       'Odisha', 'Rajasthan', 'Telangana', 'Uttar Pradesh', 'West Bengal'],
      dtype=object)

In [51]:
df_compl_R = df_compl_R[df_compl_R.State.isin(states_compl_R)]

In [52]:
df_compl_R['State'].unique()

array(['Jharkhand'], dtype=object)